In [0]:
# import basic package
import keras
from PIL import Image
import numpy as np

In [0]:
# download mnist data
from keras.datasets import mnist
(train_X, train_y), (test_X, test_y) = mnist.load_data()

In [0]:
# make data close to 0 and 1
train_X = (train_X[..., None]-127.5)/128

In [54]:
# show image
from IPython.display import display
def showX(X, rows=1):
    assert X.shape[0] % rows == 0
    int_X = (X*128+128).clip(0,255).astype('uint8')
    # N*784 -> N*28*28 -> 28*N*28 -> 28 * 28N
    int_X_reshape = int_X.reshape(rows, -1,28,28).swapaxes(1,2).reshape(28*rows,-1)
    display(Image.fromarray(int_X_reshape))

    # 訓練資料， X 的前 20 筆
showX(train_X[:10])
print(train_y)

[5 0 4 ... 5 6 8]


In [0]:
# define factor in the end to shrink
NZ = 32

In [0]:
# define discriminator
from keras.models import Sequential
from keras.layers import Conv2D, Activation, GlobalAveragePooling2D, Reshape

netD = Sequential([
    Conv2D(filters=32, kernel_size=3, strides=2, padding='same', activation='relu', input_shape=(28,28,1)),
    Conv2D(filters=32, kernel_size=3, strides=2, padding='same', activation='relu'),
    Conv2D(filters=32, kernel_size=3, strides=2, padding='valid', activation="relu"),
    Conv2D(filters=1, kernel_size=3, strides=3, padding='valid', activation="sigmoid"),
    Reshape((1,)),
])

In [81]:
# view discriminator's info
netD.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_21 (Conv2D)           (None, 14, 14, 32)        320       
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 7, 7, 32)          9248      
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 3, 3, 32)          9248      
_________________________________________________________________
conv2d_24 (Conv2D)           (None, 1, 1, 1)           289       
_________________________________________________________________
reshape_14 (Reshape)         (None, 1)                 0         
Total params: 19,105
Trainable params: 19,105
Non-trainable params: 0
_________________________________________________________________


In [0]:
# define generator
from keras.layers import Conv2DTranspose, Reshape
netG = Sequential([
    Reshape( (1,1,NZ), input_shape=(NZ,)),
    Conv2DTranspose(filters=32, kernel_size=3, strides=2, padding='valid', activation='relu'),
    Conv2DTranspose(filters=32, kernel_size=3, strides=2, padding='valid', activation='relu'),
    Conv2DTranspose(filters=32, kernel_size=3, strides=2, padding='same', activation='relu'),
    Conv2DTranspose(filters=1, kernel_size=3, strides=2, padding='same',activation="tanh")
])

In [83]:
# view generator's info
netG.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_15 (Reshape)         (None, 1, 1, 32)          0         
_________________________________________________________________
conv2d_transpose_33 (Conv2DT (None, 3, 3, 32)          9248      
_________________________________________________________________
conv2d_transpose_34 (Conv2DT (None, 7, 7, 32)          9248      
_________________________________________________________________
conv2d_transpose_35 (Conv2DT (None, 14, 14, 32)        9248      
_________________________________________________________________
conv2d_transpose_36 (Conv2DT (None, 28, 28, 1)         289       
Total params: 28,033
Trainable params: 28,033
Non-trainable params: 0
_________________________________________________________________


In [0]:
# define gan model
from keras.models import Model
GD_output = netD(netG.outputs)
netGD = Model(inputs=netG.inputs, outputs=[GD_output])

In [85]:
# view gan's info
netGD.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_15_input (InputLayer (None, 32)                0         
_________________________________________________________________
reshape_15 (Reshape)         (None, 1, 1, 32)          0         
_________________________________________________________________
conv2d_transpose_33 (Conv2DT (None, 3, 3, 32)          9248      
_________________________________________________________________
conv2d_transpose_34 (Conv2DT (None, 7, 7, 32)          9248      
_________________________________________________________________
conv2d_transpose_35 (Conv2DT (None, 14, 14, 32)        9248      
_________________________________________________________________
conv2d_transpose_36 (Conv2DT (None, 28, 28, 1)         289       
_________________________________________________________________
sequential_13 (Sequential)   (None, 1)                 19105     
Total para

In [86]:
# random create a tensor
import numpy as np
showX(netG.predict(np.random.normal( size=(10, NZ))))

In [0]:
# set up loss and output_real, out_fake and training update for discriminator
import keras.backend as K
from keras.optimizers import RMSprop, SGD, Adam

#loss_fn = lambda output, target : K.mean(K.binary_crossentropy(output, target))
loss_fn = lambda output, target : -K.mean(K.log(output+1e-12)*target+K.log(1-output+1e-12)*(1-target))

input_real = netD.inputs[0]
input_z = netG.inputs[0]

output_D_real = netD.outputs[0]
output_D_fake = netGD.outputs[0]

loss_D_real = loss_fn(output_D_real, K.ones_like(output_D_real))
loss_D_fake = loss_fn(output_D_fake, K.zeros_like(output_D_fake))
loss_D = (loss_D_real +loss_D_fake)/2

training_updates = RMSprop(lr=1e-3).get_updates(netD.trainable_weights,[],loss_D)
netD_train = K.function([input_z, input_real],[loss_D/2], training_updates)

In [0]:
# set up loss and output_real, out_fake and training update for generator
loss_G = loss_fn(output_D_fake, K.ones_like(output_D_fake))
training_updates = RMSprop(lr=1e-3).get_updates(netG.trainable_weights,[], loss_G)
netG_train = K.function([input_z], [loss_G], training_updates)

In [0]:
# define minibatch for training
def minibatch(dataset, batchsize):
  while True:
    yield dataset[np.random.choice(dataset.shape[0], batchsize, replace=False)]

In [90]:
# test for show image result
train_batch = minibatch(train_X, 10)
for i in range(5):
  X = next(train_batch)
  showX(X)

In [0]:
# train model
import time
from IPython.display import clear_output
t0 = time.time()
gen_iterations = 0
errG = 0
errG_sum = errD_sum = 0

display_iters = 100
batch_size = 32

train_batch = minibatch(train_X, batch_size)
fixed_Z = np.random.normal(scale=0.5, size=(100, NZ))
while gen_iterations < 10000: 
    X = next(train_batch)
    Z = np.random.normal(size=(batch_size, NZ))
    errD, = netD_train([Z, X])
    errD_sum +=errD
    
    #Z = np.random.normal(size=(batch_size, NZ))
    errG, = netG_train([Z])
    errG_sum += errG    
    gen_iterations+=1
    
    if gen_iterations%display_iters==0:
        clear_output(True)
        print('[%d] Loss_D: %f Loss_G: %f'
        % (gen_iterations, errD_sum/display_iters, errG_sum/display_iters), time.time()-t0)
        fakeX = netG.predict(fixed_Z)
        showX(fakeX, 10)
        errG_sum = errD_sum = 0

In [0]:
5